In [2]:
import scanpy as sc

# 读取新生成的文件
adata = sc.read_h5ad("pbmc_for_gpt_fixed.h5ad")

# 检查
print(adata)
print(f"\n细胞元数据列名: {adata.obs.columns.tolist()}")
print(f"矩阵 X 类型: {type(adata.X)}")

AnnData object with n_obs × n_vars = 7778 × 52238
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'ident'
    uns: 'X_name'
    layers: 'logcounts'

细胞元数据列名: ['orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'ident']
矩阵 X 类型: <class 'scipy.sparse._csc.csc_matrix'>


n_obs × n_vars (7778 × 52238)：你有 7,778 个细胞和 52,238 个基因。基因数较多，说明这是包含全部基因的全转录组数据，非常适合 scGPT。
矩阵 X 类型 (csc_matrix)：这是标准的稀疏矩阵格式，内存占用低，运行效率高。

layers: 'logcounts'：这说明你的标准化数据被存放在了 layers 里，而默认的 X 通常存放的是原始计数（Counts）或者转换时指定的层。


In [5]:
# 1. 习惯性将 CSC 转为 CSR，因为 Scanpy 很多函数在 CSR 下更快
adata.X = adata.X.tocsr()

# 2. 确保 obs 和 var 的索引是字符串格式（防止某些工具报错）
adata.obs_names = adata.obs_names.astype(str)
adata.var_names = adata.var_names.astype(str)

# 3. 备份原始数据到 .raw 槽位（scGPT 常用操作）
adata.raw = adata

# 4. 保存为最终版，之后 Python 分析直接读这个文件，不用再管 R 了
adata.write_h5ad("pbmc_ready_for_scGPT.h5ad")


In [6]:
import torch
import torchtext

print(f"Torch version: {torch.__version__}")
print(f"Torchtext version: {torchtext.__version__}")

# 测试 C++ 扩展是否能加载 (如果没有报错，就成功了)
from torchtext._extension import _init_extension
_init_extension()

# 尝试导入 scGPT
from scgpt.model import TransformerModel



Torch version: 2.1.0+cu121
Torchtext version: 0.16.0+cpu


In [13]:
import torch
import numpy as np
import scanpy as sc

from tdc import tdc_hf_interface
from tdc.model_server.tokenizers.scgpt import scGPTTokenizer


ModuleNotFoundError: No module named 'tdc'

In [ ]:
import torch
import os
from huggingface_hub import snapshot_download
from scgpt.model import TransformerModel
from scgpt.tokenizer.gene_tokenizer import GeneVocab

# 1. 自动从 Hugging Face 下载模型文件夹
model_dir = snapshot_download(repo_id="wanglab/scgpt-base")
print(f"模型已下载至: {model_dir}")

# 2. 加载词表 (Vocab)
vocab_path = os.path.join(model_dir, "vocab.json")
vocab = GeneVocab.from_file(vocab_path)

# 3. 实例化模型结构
# wanglab/scgpt-base 的默认架构参数如下：
model = TransformerModel(
    ntokens=len(vocab),
    d_model=512,
    nhead=8,
    d_hid=512,
    nlayers=12,
    nlayers_cls=3,
    # 如果该模型支持预测表达量，可能还需要增加其它参数，但做 Embedding 这样足够了
)

# 4. 加载权重文件 (.pt 或 .bin)
# 注意：文件名可能是 'best_model.pt' 或 'pytorch_model.bin'，请检查下载目录
ckpt_path = os.path.join(model_dir, "best_model.pt")
model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print("模型加载成功！")